In [10]:
## This notebook is for debugging the test loader for the images which are producing different prediction results with the sam model and parameters but different seed

In [18]:
# auto-load when code changes outside
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

import sys
sys.path.insert(0,'..')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument


In [19]:
#imports 
import os
import nltk
import spacy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.onnx as onnx
import torch.optim as optim
import torchvision.models as models

from src.baseline.vocabulary import Vocabulary
from src.utils import print_time, list_of_tensors_to_numpy_arr, plot_image, plot_im
from src.accuracy import Accuracy
from src.accuracy import WEIGHT_MAP, WEIGHT_MAP_ONLY_SHIELD_COLOR, WEIGHT_MAP_ONLY_CHARGE, WEIGHT_MAP_ONLY_CHARGE_COLOR
from src.baseline.coa_model import get_new_model,load_model, train_validate_test_split, init_testing_model, test_model, test_rand_image, get_training_mean_std


In [20]:
!export CUDA_VISIBLE_DEVICES=0
!export CUDA_LAUNCH_BLOCKING=1

In [21]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

device

device(type='cuda', index=0)

In [22]:
# torch.cuda.get_device_name(0)

In [23]:
# Hyperparams

embed_size=300
vocab_size = 36
attention_dim=256
encoder_dim=512
decoder_dim=512
learning_rate = 3e-4
drop_prob=0.3
ignored_idx = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hyper_params = {'embed_size': embed_size,
                'attention_dim': attention_dim,
                'encoder_dim': encoder_dim,
                'decoder_dim': decoder_dim,
                'vocab_size': vocab_size
              }


In [24]:
data_location =  '/home/salnabulsi/coat-of-arms/data/new/'

root_folder_images = data_location + '/resized'

# test_caption_file = data_location + '/test_captions_faulty.txt'

test_caption_file = data_location + '/test_captions_psumsq.txt'
df = pd.read_csv(test_caption_file)

print("There are {} test images".format(len(df)))

There are 29 test images


In [31]:
EXP_FOLDER_PATH = '/home/space/datasets/COA/experiments/' 
run_path   = EXP_FOLDER_PATH + 'run-03-12-2023-19:37:19/'
model_file = 'baseline-model-03-13-2023-11:47:01.pth'
# BATCH_SIZE = 5
NUM_WORKER = 2 #### this needs multi-core
freq_threshold = 5
batch_size = 3


vocab = Vocabulary(freq_threshold)
vocab.stoi = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'lion': 4, 'rampant': 5, 'passt': 6, 'guard': 7, 'head': 8, 'lions': 9, 'cross': 10, 'molxine': 11, 'patonce': 12, 'eagle': 13, 'doubleheaded': 14, 'eagles': 15, 'a': 16, 'b': 17, 'o': 18, 's': 19, 'g': 20, 'e': 21, 'v': 22, '1': 23, '2': 24, '3': 25, '4': 26, '5': 27, '6': 28, '7': 29, '8': 30, '9': 31, '10': 32, '11': 33, 'border': 34, '&': 35}
vocab.itos = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>', 4: 'lion', 5: 'rampant', 6: 'passt', 7: 'guard', 8: 'head', 9: 'lions', 10: 'cross', 11: 'moline', 12: 'patonce', 13: 'eagle', 14: 'doubleheaded', 15: 'eagles', 16: 'a', 17: 'b', 18: 'o', 19: 's', 20: 'g', 21: 'e', 22: 'v', 23: '1', 24: '2', 25: '3', 26: '4', 27: '5', 28: '6', 29: '7', 30: '8', 31: '9', 32: '10', 33: '11', 34: 'border', 35: '&'}



In [32]:
!export CUDA_LAUNCH_BLOCKING=1
!export CUDA_VISIBLE_DEVICES=2

### Get the mean and std 
This was calculated and saved in training phase in files

In [33]:
mean,std = (torch.tensor(0.2900), torch.tensor(0.3396))
mean,std

(tensor(0.2900), tensor(0.3396))

### Init the test model

In [60]:
test_loader, test_dataset = init_testing_model(test_caption_file, 
                                               root_folder_images, 
                                               mean, std,
                                               NUM_WORKER,
                                               vocab,
                                               batch_size, 
                                               device, 
                                               pin_memory=False)



In [61]:
dataiter = iter(test_loader)
next(dataiter)

# torch.Size([3, 224, 224]) with crop


(tensor([[[[1.8828, 1.8828, 1.8020,  ..., 1.7558, 1.7905, 1.8136],
           [1.8828, 1.8597, 1.8251,  ..., 1.8597, 1.8944, 1.9406],
           [1.8944, 1.8366, 1.8366,  ..., 1.8713, 1.8944, 1.9290],
           ...,
           [1.8597, 1.8828, 1.8713,  ..., 1.7327, 1.6519, 1.3516],
           [1.8597, 1.8713, 1.8597,  ..., 0.3355, 0.3932, 0.3816],
           [1.8713, 1.8828, 1.8597,  ..., 1.4440, 1.5133, 1.4440]],
 
          [[1.7905, 1.7905, 1.7096,  ..., 1.5249, 1.5018, 1.4787],
           [1.7558, 1.7327, 1.6981,  ..., 1.6519, 1.6403, 1.6172],
           [1.7212, 1.6634, 1.6865,  ..., 1.6981, 1.6981, 1.6750],
           ...,
           [1.6519, 1.6865, 1.6981,  ..., 1.6288, 1.5480, 1.2477],
           [1.6519, 1.6750, 1.6865,  ..., 0.2315, 0.2893, 0.2777],
           [1.6634, 1.6865, 1.6865,  ..., 1.3401, 1.4094, 1.3401]],
 
          [[1.5480, 1.5480, 1.4671,  ..., 1.2477, 1.2708, 1.3170],
           [1.5249, 1.5018, 1.4671,  ..., 1.3747, 1.3978, 1.4556],
           [1.5018, 1.44

In [ ]:
dataiter = iter(test_loader)
next(dataiter)


In [80]:
dataiter = iter(test_loader)
next(dataiter)


(tensor([[[[ 1.7558,  1.7558,  1.8020,  ...,  1.7905,  1.7905,  1.8136],
           [ 1.8020,  1.8136,  1.8251,  ...,  1.8136,  1.8136,  1.8251],
           [ 1.8597,  1.8597,  1.8597,  ...,  1.8251,  1.8251,  1.8366],
           ...,
           [ 1.8020,  1.8020,  1.8020,  ...,  1.7558,  1.7558,  1.7789],
           [ 1.8136,  1.8136,  1.7905,  ...,  1.7674,  1.7674,  1.7905],
           [ 1.8251,  1.8136,  1.8020,  ...,  1.7443,  1.7674,  1.7789]],
 
          [[ 1.6057,  1.6057,  1.6519,  ...,  1.6865,  1.6865,  1.6981],
           [ 1.6519,  1.6634,  1.6750,  ...,  1.7096,  1.7096,  1.7096],
           [ 1.7096,  1.7096,  1.7096,  ...,  1.7212,  1.7212,  1.7212],
           ...,
           [ 1.6519,  1.6519,  1.6519,  ...,  1.6288,  1.6172,  1.6403],
           [ 1.6634,  1.6634,  1.6403,  ...,  1.6519,  1.6519,  1.6634],
           [ 1.6750,  1.6634,  1.6519,  ...,  1.6288,  1.6403,  1.6519]],
 
          [[ 1.3863,  1.3863,  1.4325,  ...,  1.5018,  1.4902,  1.4902],
           [ 

In [25]:
seed = 123456
random.seed(seed)     # python random generator
np.random.seed(seed)  # numpy random generator

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


NameError: name 'random' is not defined

In [ ]:
model_path= f"{run_path}/{model_file}"
model, optimizer, loss, criterion = load_model(model_path, 
                                hyper_params, 
                                learning_rate,
                                drop_prob, 
                                ignored_idx,
                                pretrained=True)


In [26]:
test_losses, accuracy_test_list, image_names_list, predictions_list, acc_test_score, test_loss = test_model(model, 
                                                                        criterion,
                                                                        test_loader, 
                                                                        test_dataset, 
                                                                        vocab_size, 
                                                                        device,
                                                                        run_path,
                                                                        True,
                                                                        WEIGHT_MAP)    


test function image_file_names ['3837_A S lion rampant.jpg', '22874_O G cross.jpg', '5096_B O lion.jpg', '22337_G O lion rampant.jpg', '3551_G A eagle.jpg', '586_S O lion rampant.jpg', '22910_O B cross.jpg', '18847_B A eagle.jpg', '19245_O B lion rampant.jpg', '7131_A G lion rampant.jpg', '6736_O B lion rampant.jpg', '1537_A S eagle doubleheaded.jpg', '17591_S A eagle.jpg', '6384_A G cross moline.jpg', '18220_A S lion rampant.jpg', '7598_O G lion rampant.jpg', '4150_A G eagle doubleheaded.jpg', '32013_B A lion rampant.jpg', '10757_B O eagle.jpg', '7037_A G cross.jpg', '16852_A B eagle doubleheaded.jpg', '27618_G A lion rampant.jpg', '22323_O S cross moline.jpg', '561_B A cross.jpg', '22122_O S cross.jpg', '4660_G O eagle.jpg', '6679_O S eagle.jpg', '7350_A V lion rampant.jpg', '715_O S lion rampant.jpg'] in batch 0


/home/salnabulsi/.thesis-py38/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


image_file_name, acc_image, acc_charge_only,acc_image_charge_color, acc_shield_only: 3837_A S lion rampant.jpg, 1.0, 1.0, 1, 1.0, a s lion rampant
image_file_name, acc_image, acc_charge_only,acc_image_charge_color, acc_shield_only: 22874_O G cross.jpg, 1.0, 1.0, 1, 1.0, o g cross
image_file_name, acc_image, acc_charge_only,acc_image_charge_color, acc_shield_only: 5096_B O lion.jpg, 0.67, 1.0, 1, 0.0, g o lion rampant
image_file_name, acc_image, acc_charge_only,acc_image_charge_color, acc_shield_only: 22337_G O lion rampant.jpg, 0.33, 1.0, 0, 0.0, a g lion rampant
image_file_name, acc_image, acc_charge_only,acc_image_charge_color, acc_shield_only: 3551_G A eagle.jpg, 1.0, 1.0, 1, 1.0, g a eagle
image_file_name, acc_image, acc_charge_only,acc_image_charge_color, acc_shield_only: 586_S O lion rampant.jpg, 0.33, 1.0, 0, 0.0, a a lion rampant
image_file_name, acc_image, acc_charge_only,acc_image_charge_color, acc_shield_only: 22910_O B cross.jpg, 0.33, 1.0, 0, 0.0, a g cross
image_file_name

## Visualizing the attentions

In [87]:
# torch.cuda.empty_cache()
test_rand_image(model,test_dataset,test_loader,device)

ValueError: too many values to unpack (expected 4)